# 🌴 Asistente Turístico de Tenerife - RAG Demo

## 1. Configuración del Entorno
Cargamos las librerías necesarias y verificamos la API Key.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ Key cargada: {api_key[:5]}...")
else:
    print("❌ Error de Key")

✅ Key cargada: sk-pr...


## 2. Validación de Conectividad con OpenAI
Probamos que la llave funcione y el modelo responda.

In [2]:
from langchain_openai import ChatOpenAI

# Inicializamos el modelo
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Prueba 'Holística'
try:
    response = llm.invoke("Di 'Hola Tenerife' si me escuchas.")
    print(f"🤖 Respuesta Modelo: {response.content}")
except Exception as e:
    print(f"❌ Error de Conexión: {e}")

🤖 Respuesta Modelo: Hola Tenerife. ¿En qué puedo ayudarte hoy?


## 3. Fase 2: Ingesta y Vector Store 📚
Subimos el PDF `TENERIFE.pdf` a OpenAI para crear un Vector Store gestionado.

In [3]:
import os
import sys
import openai
from openai import OpenAI
from pathlib import Path

# 1. DIAGNÓSTICO DE ENTORNO (CRÍTICO)
print(f"🐍 Python Executable: {sys.executable}")
if "llm-env" not in sys.executable:
    print("⚠️ ADVERTENCIA: No parece que estés usando el entorno virtual 'llm-env'. Verifica tu Kernel.")

print(f"📦 OpenAI Version: {openai.__version__}")

client = OpenAI()

pdf_path = Path("../data/raw/TENERIFE.pdf")

if not pdf_path.exists():
    print(f"❌ No encontrado: {pdf_path}")
else:
    print(f"✅ Archivo encontrado: {pdf_path}")

    # 2. ADAPTADOR UNIVERSAL DE VECTOR STORES
    try:
        vs_manager = None
        
        # Intento 1: Ruta estándar V2 Beta
        if hasattr(client, 'beta') and hasattr(client.beta, 'vector_stores'):
            vs_manager = client.beta.vector_stores
            print("✅ Conectado vía client.beta.vector_stores")
            
        # Intento 2: Ruta raíz (posible cambio futuro o versión diferente)
        elif hasattr(client, 'vector_stores'):
            vs_manager = client.vector_stores
            print("✅ Conectado vía client.vector_stores")

        if vs_manager:
            vector_store = vs_manager.create(name="Tenerife Guide Store")
            print(f"📦 Vector Store creado: {vector_store.id}")
            
            with open(pdf_path, "rb") as f:
                file_batch = vs_manager.file_batches.upload_and_poll(
                    vector_store_id=vector_store.id, files=[f]
                )
            print(f"📄 Estado de carga: {file_batch.status}")
            print(f"🔢 Archivos procesados: {file_batch.file_counts}")
        else:
            # FALLO TOTAL: Imprimir introspección para depurar
            print("❌ ERROR: No se encuentra 'vector_stores' en el cliente OpenAI.")
            print("🔍 Depuración de atributos disponibles:")
            print(f"   client dir: {[x for x in dir(client) if 'vector' in x or 'beta' in x]}")
            if hasattr(client, 'beta'):
                print(f"   client.beta dir: {dir(client.beta)}")
            else:
                print("   client.beta NO existe.")
            raise AttributeError("No se pudo acceder a Vector Stores con esta versión de librería/entorno.")

    except Exception as e:
        print(f"❌ Excepción: {e}")

🐍 Python Executable: f:\development\Development\Master IA\LLM-large-language-models-entrega-tarea-final\llm-env\Scripts\python.exe
📦 OpenAI Version: 2.15.0
✅ Archivo encontrado: ..\data\raw\TENERIFE.pdf
✅ Conectado vía client.vector_stores
📦 Vector Store creado: vs_6966683feec88191b28f904269d7db29
📄 Estado de carga: completed
🔢 Archivos procesados: FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


## 4. Fase 3: Configuración del Agente RAG 🤖
Configuramos el agente con capacidad de usar el Vector Store para responder preguntas sobre Tenerife.
**Nota**: Asegúrate de haber ejecutado la Fase 2 para tener la variable `vector_store` disponible.

In [4]:
# ASEGURAR DEPENDENCIAS EN EL KERNEL ACTUAL
# Ejecuta esta celda si obtienes 'ModuleNotFoundError'
%pip install -q -U openai-agents nest_asyncio openai
print("✅ Librerías instaladas/verificadas en el Kernel actual.")

Note: you may need to restart the kernel to use updated packages.
✅ Librerías instaladas/verificadas en el Kernel actual.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


> ⚠️ **IMPORTANTE: REINICIA EL KERNEL AHORA** ⚠️
>
> Acabas de instalar nuevas librerías. Para que Python las reconozca, debes ir al menú **Kernel > Restart Kernel**.
> Luego, puedes continuar ejecutando la celda de abajo (la que da error `ModuleNotFoundError`).

In [5]:
import nest_asyncio
nest_asyncio.apply()

from openai_agents.agents import Agent, Runner
from openai_agents.tools import FileSearchTool

# Intentamos obtener el ID del store creado en la celda anterior.
# Si se reinició el kernel, se puede usar el ID hardcodeado de la ejecución previa.
try:
    vector_store_id = vector_store.id
except NameError:
    print("⚠️ 'vector_store' no está en memoria. Usando ID precargado (asegúrate que sea válido).")
    vector_store_id = "vs_69661c161c5881919eadf77744958c70"

# 1. Definir la herramienta de búsqueda de archivos
file_search_tool = FileSearchTool(vector_store_id=vector_store_id)

# 2. Instrucciones del Agente (System Prompt)
INSTRUCTIONS = """
Eres un experto guía turístico de Tenerife.
Usa la herramienta file_search para encontrar información en la guía PDF adjunta.
Si la información no está en el PDF, dilo claramente. No inventes respuestas.
Responde siempre en español, con un tono amable y entusiasta.
"""

# 3. Crear el Agente
agent = Agent(
    name="TenerifeGuide",
    model="gpt-3.5-turbo",
    instructions=INSTRUCTIONS,
    tools=[file_search_tool]
)

print(f"🤖 Agente '{agent.name}' configurado satisfactoriamente.")

ModuleNotFoundError: No module named 'openai_agents'

### Prueba Rápida del Agente
Verificamos que el agente pueda recuperar información del PDF.

In [ ]:
async def test_agent():
    result = await Runner.run(agent, "¿Cuáles son las mejores playas del sur de Tenerife?")
    print(f"❓ Pregunta: ¿Cuáles son las mejores playas del sur de Tenerife?")
    print(f"💡 Respuesta: {result.final_response}")

await test_agent()